In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Load the data
df = pd.read_csv('../Data/data_5KNN.csv')
# Transform it into numpy array
data = df.to_numpy()

# Get the variables
X = data[:,1:]
y = data[:,0]

# Shape
[n, p] = np.shape(X)

In [3]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoLars, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [4]:
# 1. Outer K-Fold
K_outer = 5
outer_cv = KFold(n_splits=K_outer)

# Define a range of alphas to try
lambda_values = np.logspace(-3, 2, 100)
l1_ratios = np.linspace(0.1, 0.95, 19)   # Mix ratio between Lasso (L1) and Ridge (L2)

# Store the outer test RMSE for each model type
ols_outer_rmse = []
ridge_outer_rmse = []
lasso_outer_rmse = []
elasticnet_outer_rmse = []

# Store the best models from each fold
best_ols_models = []
best_ridge_models = []
best_lasso_models = []
best_elasticnet_models = []

# Store the best parameters from each fold
best_ridge_params = []
best_lasso_params = []
best_elasticnet_params = []

# 2. Inner CV Loop
for train_idx, test_idx in outer_cv.split(X):
    # Split data into inner training and outer test sets
    X_train_outer, X_test_outer = X[train_idx], X[test_idx]
    y_train_outer, y_test_outer = y[train_idx], y[test_idx]

    # 2.0 Inner CV: OLS (Linear Regression)
    # Define a pipeline that includes normalization and OLS regression
    ols_pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('ols', LinearRegression())    # Apply OLS Regression
    ])
    
    # Fit OLS model on the outer training set
    ols_pipeline.fit(X_train_outer, y_train_outer)
    
    # Calculate OLS RMSE using cross-validation on the outer training set
    ols_cv = GridSearchCV(
        estimator=ols_pipeline,
        param_grid={},  # No hyperparameters to tune for OLS
        scoring="neg_mean_squared_error",
        cv=KFold(n_splits=5)
    )
    ols_cv.fit(X_train_outer, y_train_outer)
    
    # Obtain the model, score (MSE) and RMSE
    best_ols_model = ols_pipeline  # The fitted pipeline
    best_ols_models.append(best_ols_model)
    best_ols_score = ols_cv.best_score_  # negative MSE
    best_ols_rmse = np.sqrt(-best_ols_score)  # RMSE

    # 2.1 Inner CV: RIDGE
    # Define a pipeline that includes normalization and regression to indtroduce in the GridSearchCV
    ridge_pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('ridge', Ridge())             # Apply Ridge Regression
    ])
    
    # We need a dictionary as an input for the parameters
    ridge_param_grid = {'ridge__alpha' : lambda_values}

    # Perform GridSearchCV to find the best lambda (alpha in this case)
    ridge_cv = GridSearchCV(
        estimator=ridge_pipeline,
        param_grid=ridge_param_grid,
        scoring="neg_mean_squared_error",
        cv=KFold(n_splits=5)
    )
    ridge_cv.fit(X_train_outer, y_train_outer)  # it is using the normalized data stablished in the pipeline
    
    # Obtain the best parameters and the corresponding model
    best_lambda_ridge = ridge_cv.best_params_['ridge__alpha']
    best_ridge_params.append(best_lambda_ridge)

    # Obtain the best model, score (MSE) and RMSE
    best_ridge_model = ridge_cv.best_estimator_  # pipeline refit on entire outer-training set
    best_ridge_models.append(best_ridge_model)
    best_ridge_score = ridge_cv.best_score_      # negative MSE (the higher, the better)
    best_ridge_rmse = np.sqrt(-best_ridge_score)  # turn negative MSE into positive MSE and obtain RMSE

    # 2.2 Inner CV: LASSO LARS
    # Define a pipeline that includes normalization and regression to indtroduce in the GridSearchCV
    lasso_lars_pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('lasso_lars', LassoLars())             # Apply Lasso LARS Regression
    ])
    
    # We need a dictionary as an input for the parameters
    lasso_lars_param_grid = {'lasso_lars__alpha' : lambda_values}

    # Perform GridSearchCV to find the best lambda (alpha in this case)
    lasso_lars_cv = GridSearchCV(
        estimator=lasso_lars_pipeline,
        param_grid=lasso_lars_param_grid,
        scoring="neg_mean_squared_error",
        cv=KFold(n_splits=5)
    )

    lasso_lars_cv.fit(X_train_outer, y_train_outer)

    # Obtain the best parameters and the corresponding model
    best_lambda_lasso_lars = lasso_lars_cv.best_params_['lasso_lars__alpha']
    best_lasso_params.append(best_lambda_lasso_lars)

    # Obtain the best model, score (MSE) and RMSE
    best_lasso_model = lasso_lars_cv.best_estimator_
    best_lasso_models.append(best_lasso_model)
    best_lasso_score = lasso_lars_cv.best_score_
    best_lasso_rmse = np.sqrt(-best_lasso_score)

    # 2.3 Inner CV: ELASTICNET
    # Define a pipeline that includes normalization and regression to introduce in GridSearchCV
    elasticnet_pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('elasticnet', ElasticNet(max_iter=20000))  # Apply ElasticNet with a high iteration limit
    ])

    # We need a dictionary as an input for the parameters
    elasticnet_param_grid = {
        'elasticnet__alpha': lambda_values,
        'elasticnet__l1_ratio': l1_ratios  # The mix ratio between Lasso and Ridge
    }

    # Perform GridSearchCV to find the best lambda (alpha in this case)
    elasticnet_cv = GridSearchCV(
        estimator=elasticnet_pipeline,
        param_grid=elasticnet_param_grid,
        scoring="neg_mean_squared_error",
        cv=KFold(n_splits=5),
        n_jobs=-1
    )
    elasticnet_cv.fit(X_train_outer, y_train_outer)

    # Obtain the best parameters and the corresponding model
    best_lambda_elasticnet = elasticnet_cv.best_params_['elasticnet__alpha']
    best_l1_ratio_elasticnet = elasticnet_cv.best_params_['elasticnet__l1_ratio']
    best_elasticnet_params.append((best_lambda_elasticnet, best_l1_ratio_elasticnet))

    # Obtain the best model, score (MSE) and RMSE
    best_elasticnet_model = elasticnet_cv.best_estimator_
    best_elasticnet_models.append(best_elasticnet_model)
    best_elasticnet_score = elasticnet_cv.best_score_
    best_elasticnet_rmse = np.sqrt(-best_elasticnet_score)

    # 2.4 Evaluate all models on the Outer Test Fold
    # OLS prediction
    y_pred_ols = best_ols_model.predict(X_test_outer)
    ols_rmse = root_mean_squared_error(y_test_outer, y_pred_ols)
    ols_outer_rmse.append(ols_rmse)
    
    # Ridge prediction
    y_pred_ridge = best_ridge_model.predict(X_test_outer)
    ridge_rmse = root_mean_squared_error(y_test_outer, y_pred_ridge)
    ridge_outer_rmse.append(ridge_rmse)
    
    # Lasso prediction
    y_pred_lasso = best_lasso_model.predict(X_test_outer)
    lasso_rmse = root_mean_squared_error(y_test_outer, y_pred_lasso)
    lasso_outer_rmse.append(lasso_rmse)
    
    # ElasticNet prediction
    y_pred_elasticnet = best_elasticnet_model.predict(X_test_outer)
    elasticnet_rmse = root_mean_squared_error(y_test_outer, y_pred_elasticnet)
    elasticnet_outer_rmse.append(elasticnet_rmse)
    
    # Print results for this fold
    print(f"Outer RMSE - OLS: {ols_rmse:.4f}, Ridge: {ridge_rmse:.4f}, Lasso: {lasso_rmse:.4f}, ElasticNet: {elasticnet_rmse:.4f}")

# 3. Final Estimate of Generalization Error (in terms of RMSE) for each model
ols_cv_rmse = np.mean(ols_outer_rmse)
ridge_cv_rmse = np.mean(ridge_outer_rmse)
lasso_cv_rmse = np.mean(lasso_outer_rmse)
elasticnet_cv_rmse = np.mean(elasticnet_outer_rmse)

print("\nAverage RMSE across all folds:")
print(f"OLS: {ols_cv_rmse:.4f}")
print(f"Ridge: {ridge_cv_rmse:.4f}")
print(f"Lasso: {lasso_cv_rmse:.4f}")
print(f"ElasticNet: {elasticnet_cv_rmse:.4f}")

# Determine the best overall model
model_comparison = {
    'OLS': ols_cv_rmse,
    'Ridge': ridge_cv_rmse,
    'Lasso': lasso_cv_rmse,
    'ElasticNet': elasticnet_cv_rmse
}
best_model_overall = min(model_comparison, key=model_comparison.get)
print(f"\nBest model overall: {best_model_overall} with RMSE = {model_comparison[best_model_overall]:.4f}")

# Display the parameters for each model across folds
print("\nRidge alpha parameters across folds:", [f"{x:.2f}" for x in best_ridge_params])
print("Lasso alpha parameters across folds:", [f"{x:.2f}" for x in best_lasso_params])
print("ElasticNet parameters across folds (alpha, l1_ratio):", [f"({a:.2f}, {lr:.2f})" for a, lr in best_elasticnet_params])

# Create a new model with averaged parameters from the best model type
if best_model_overall == 'OLS':
    final_model = Pipeline([
        ('scaler', StandardScaler()),
        ('ols', LinearRegression())
    ])
    print(f"\nCreating final OLS model")
    
elif best_model_overall == 'Ridge':
    avg_alpha = np.mean(best_ridge_params)
    final_model = Pipeline([
        ('scaler', StandardScaler()),
        ('ridge', Ridge(alpha=avg_alpha))
    ])
    print(f"\nCreating final Ridge model with averaged alpha = {avg_alpha:.4f}")
    
elif best_model_overall == 'Lasso':
    avg_alpha = np.mean(best_lasso_params)
    final_model = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso_lars', LassoLars(alpha=avg_alpha))
    ])
    print(f"\nCreating final Lasso model with averaged alpha = {avg_alpha:.4f}")
    
elif best_model_overall == 'ElasticNet':
    avg_alpha = np.mean([params[0] for params in best_elasticnet_params])
    avg_l1_ratio = np.mean([params[1] for params in best_elasticnet_params])
    final_model = Pipeline([
        ('scaler', StandardScaler()),
        ('elasticnet', ElasticNet(alpha=avg_alpha, l1_ratio=avg_l1_ratio, max_iter=20000))
    ])
    print(f"\nCreating final ElasticNet model with averaged alpha = {avg_alpha:.4f} and l1_ratio = {avg_l1_ratio:.4f}")

# Fit the final model on the entire dataset
final_model.fit(X, y)

# Evaluate the final model on the entire dataset (in-sample error)
y_pred = final_model.predict(X)
final_rmse = root_mean_squared_error(y, y_pred)
print(f"Final model in-sample RMSE: {final_rmse:.4f}")

Outer RMSE - OLS: 42.1190, Ridge: 37.8464, Lasso: 28.3587, ElasticNet: 34.3942
Outer RMSE - OLS: 55.3581, Ridge: 29.9758, Lasso: 20.3010, ElasticNet: 25.2484
Outer RMSE - OLS: 48.5622, Ridge: 44.8163, Lasso: 33.2612, ElasticNet: 39.8793
Outer RMSE - OLS: 34.3358, Ridge: 31.8857, Lasso: 31.9054, ElasticNet: 32.1081
Outer RMSE - OLS: 46.7073, Ridge: 32.9211, Lasso: 26.9467, ElasticNet: 29.1768

Average RMSE across all folds:
OLS: 45.4165
Ridge: 35.4891
Lasso: 28.1546
ElasticNet: 32.1614

Best model overall: Lasso with RMSE = 28.1546

Ridge alpha parameters across folds: ['7.74', '9.77', '8.70', '4.86', '6.89']
Lasso alpha parameters across folds: ['3.05', '2.15', '1.92', '1.52', '2.42']
ElasticNet parameters across folds (alpha, l1_ratio): ['(0.34, 0.95)', '(0.95, 0.95)', '(1.52, 0.95)', '(0.67, 0.95)', '(0.53, 0.95)']

Creating final Lasso model with averaged alpha = 2.2132
Final model in-sample RMSE: 20.2969
